https://keras.io/examples/lstm_seq2seq/

## Sequence to sequence example in Keras (character-level).

This script demonstrates how to implement a basic character-level sequence-to-sequence model. We apply it to translating short English sentences into short French sentences, character-by-character. Note that it is fairly unusual to do character-level machine translation, as word-level models are more common in this domain.

In [3]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import random
from typing import List, Dict, Tuple

In [10]:
path = 'data/sent-comp.train.tsv'
text = open(path, encoding='utf-8').read()
print('Corpus length:', len(text))

Corpus length: 64974


In [2]:
print(type((1,2)))

<class 'tuple'>


In [7]:
def read_pairs(fn: str)-> List[Tuple]:
    res = []
    with open(fn, 'rt', encoding='utf-8') as f:
        next(f)  # skip header
        for line in f:
            line = line.strip()
            original, compressed = line.split('\t')
            res.append((original,compressed))
    return res

In [10]:
samples = read_pairs('data/sent-comp.train.tsv')
print(f"Total samples: {len(samples)}")
for s in samples[:10]:
    print(s)

Total samples: 200000
('Serge Ibaka -- the Oklahoma City Thunder forward who was born in the Congo but played in Spain -- has been granted Spanish citizenship and will play for the country in EuroBasket this summer, the event where spots in the 2012 Olympics will be decided.', 'Serge Ibaka has been granted Spanish citizenship and will play in EuroBasket.')
('MILAN -Catania held Roma to a 1-1 draw in Serie A on Wednesday as the teams played out the remaining 25 minutes of a game that was called off last month.', 'Catania held Roma to a 1 1 draw in Serie A.')
('State Street Corporation, a provider of investment servicing, investment management and investment research and trading services, has launched a new investment servicing solution to support small to mid-sized asset managers with their investment operations needs.', 'State Street Corporation, has launched a new investment servicing solution.')
('Massey Energy, the fourth largest coal company in the country, could have its corporate

In [26]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
start_sample = 10000

In [27]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
for s in samples[start_sample: start_sample+min(num_samples, len(samples))]:
    input_text, target_text = s
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 124
Number of unique output tokens: 105
Max sequence length for inputs: 1004
Max sequence length for outputs: 187


In [28]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [29]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=5,
          validation_split=0.2)
# Save model
# model.save('data/s2s.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 335s 42ms/step - loss: 0.9446 - val_loss: 0.9021
Epoch 2/5
8000/8000 [==============================] - 325s 41ms/step - loss: 0.8282 - val_loss: 0.7936
Epoch 3/5
8000/8000 [==============================] - 327s 41ms/step - loss: 0.7561 - val_loss: 0.7459
Epoch 4/5
8000/8000 [==============================] - 330s 41ms/step - loss: 0.7181 - val_loss: 0.7181
Epoch 5/5
8000/8000 [==============================] - 352s 44ms/step - loss: 0.6891 - val_loss: 0.6902


C:\Users\ssotn\Anaconda3\envs\nlp\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_3/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [30]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [31]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Whitehead discusses protecting individual rights for everyone no matter their political affiliation.
Decoded sentence: A man has been the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the t
-
Input sentence: Cougars great and former Saints player Steve Gleason should be released from the hospital Tuesday afternoon.
Decoded sentence: A man has been the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the t
-
Input sentence: What do newspaper front pages tell us about the status of women in journalism and public life today?
Decoded sentence: A man has been the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the t
-
Input senten

-
Input sentence: US new home sales surged in July, rising 9.6 percent over the previous month's rate, the US Commerce Department reported Wednesday.
Decoded sentence: A man has been the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the t
-
Input sentence: London Mayor Boris Johnson said he was lost in admiration at the achievements of Team GB, after receiving the Olympic flag in Beijing.
Decoded sentence: A man has been the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the t
-
Input sentence: HILLARY Clinton will visit Indonesia on her first overseas trip as US Secretary of State because she wants to ``reach out'' to the Muslim world.
Decoded sentence: A man has been the the the the the the the the the the the the the the the the the the the the the the

-
Input sentence: Three people killed in a fiery head-on car crash near Florence have been identified.
Decoded sentence: A man has been the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the t
-
Input sentence: Inter Milan striker Adriano has escaped a possible two-game ban after scoring the opener with his arm in Sunday's 2-1 win over AC Milan.
Decoded sentence: A man has been the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the t
-
Input sentence: Clashes erupted again between the army and M23 rebels in eastern Democratic Republic of Congo, the restive region where fighting has been flaring all week, the two sides said.
Decoded sentence: A man has been the the the the the the the the the the the the the the the the the the the the the the the the the t

-
Input sentence: Champion spinner Muttiah Muralitharan Saturday said the Bangladeshi players don't know how to play long innings although there are many talented players.
Decoded sentence: A man has been the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the t
-
Input sentence: No one was injured in a car fire Monday morning east of Port Clinton, according to the chief of the Port Clinton Fire Department.
Decoded sentence: A man has been the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the t
-
Input sentence: Two hundred years after the birth of Abraham Lincoln, we still seek him in words and in action, in lessons from his life, in reminders of his legend -- and in pilgrimages to the temple built in his honor.
Decoded sentence: A man has been the the th

-
Input sentence: Bindi makes her feature film acting debut in ``Free Willy:
Decoded sentence: A man has been the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the t
-
Input sentence: British public sector employees will pay more in their pension fund in the emergency budget to be unveiled by Chancellor George Osborne on Tuesday.
Decoded sentence: A man has been the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the t
-
Input sentence: The Canadian dollar opened at 77.51 cents US this morning, down 0.24 of a cent from Wednesday's close.
Decoded sentence: A man has been the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the t
-
Input sentenc

In [25]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[-seq_index-2: -seq_index-1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[-seq_index-2])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Kate Hudson has revealed that she once stalked a man she was desperate to date.
Decoded sentence: A man is to a conter in the seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a s
-
Input sentence: MADRID, Spain Real Madrid has fired Bernd Schuster two days after the German coach said it would be ''impossible'' for the two-time defending Spanish champion to win against Barcelona next Saturday.
Decoded sentence: The Count has been seated a to a seate of the seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare
-
Input sentence: The Food Bank for Monterey County has received a $5,000 grant from the Barnet Segal Charitable Trust for unrestricted support of its emergency supplement

-
Input sentence: The last well has cracked in Guba mass burial site, which is the proof of the genocide committed against Azerbaijanis by Armenians in 1918, the bones in it are crumbling, head of the Guba genocide expedition, researcher of Archeology and Ethnography Institute of Azerbaijan National Academy of Sciences Gahraman Agayev told APA.
Decoded sentence: The Cont of the seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a se
-
Input sentence: A group of young missionaries from the United States is currently conducting outreach work in the Cayo district.
Decoded sentence: A man was sented a to a seate of the seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a sea
-
Input 

-
Input sentence: Volunteers were honored Wednesday for their efforts to reduce recidivism in Atlantic County.
Decoded sentence: A man was sented a to a seate of the seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a sea
-
Input sentence: Three US senators whose helicopters made emergency landings today in the mountains of Afghanistan are all OK.
Decoded sentence: The Cont of the seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a se
-
Input sentence: ``After all, Ajit Pawar is treading the path of his uncle Sharad Pawar, who had once vowed never to return to Congress fold but he did return. Similarly, when Ajit has to face the trial by fire, he will beat a sudden retreat and 

-
Input sentence: MOSCOW, Nov 23 Russian soccer fans have started a petition to retain Guus Hiddink as national coach despite the team's failure to qualify for next year's World Cup finals.
Decoded sentence: The Count has been seated a to a seate of the seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare
-
Input sentence: Commander in Chief Fidel Castro sent his congratulations to the Cuban medical brigades ``Henry Reeve'' --which fulfilled a mission in Chile after the February earthquake- and ``Todos con Voz,'' that carried out a study on the disabled in Nicaragua.
Decoded sentence: A man will seat has been seated a to a seate of the seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a
-
Input sentence: President I

-
Input sentence: 49ers Vernon Davis Dunk Gets Rejected By Crossbar After Touchdown Vernon Davis Dunk Gets Rejected By Crossbar After Touchdown Vernon Davis Dunk Gets Rejected By Crossbar After Touchdown
Decoded sentence: A man is to a conter in the seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a s
-
Input sentence: The 10 best ghosts caught in movies are a lively bunch.
Decoded sentence: The Cont of the seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a se
-
Input sentence: Depa, the Dubai based outfitter for the Burj Khalifa, is bracing itself for an 'extremely tough' 2010 in order to cope with difficult market conditions, its chief executive officer said on T

-
Input sentence: ``My sense is people are going to come out in significant numbers,'' Branstad told anchor Bret Baier of FOX News Channel in an interview inside the Iowa Statehouse.
Decoded sentence: A man is to a conter in the seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a s
-
Input sentence: The housing market maintained its strong upward trend in February, helped by improving household incomes, low mortgage rates and constrained supply, Westpac said in an analysis of today's real estate data for February.
Decoded sentence: The Cont of the seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a seare to a se


In [32]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=5,
          validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 383s 48ms/step - loss: 0.6651 - val_loss: 0.6690
Epoch 2/5
8000/8000 [==============================] - 431s 54ms/step - loss: 0.6442 - val_loss: 0.6512
Epoch 3/5
8000/8000 [==============================] - 381s 48ms/step - loss: 0.6266 - val_loss: 0.6350
Epoch 4/5
8000/8000 [==============================] - 379s 47ms/step - loss: 0.6117 - val_loss: 0.6216
Epoch 5/5
8000/8000 [==============================] - 357s 45ms/step - loss: 0.5986 - val_loss: 0.6090


In [33]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[-seq_index-2: -seq_index-1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[-seq_index-2])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Mobile website builder Akmin's mobiSiteGalore announced on Monday that it has launched version 4.0 of its mobile website builder solution.
Decoded sentence: A man was been ared to a proper to a contrice and the reash to a contrice and the reare the record to a proper to a contrice and the reare the record to a proper to a contrice and the rear
-
Input sentence: No. 3 Kentucky and No. 4 Louisville renew their rivalry in the most anticipated yearly game in the Commonwealth on Saturday at Rupp Arena.
Decoded sentence: A man was been ared to a proper to a contrice and the reash to a contrice and the reare the record to a proper to a contrice and the reare the record to a proper to a contrice and the rear
-
Input sentence: Richard S. Johnson birdied three of his last four holes to shoot a 6-under-par 64 Sunday and win the US Bank Championship for his first US PGA Tour victory.
Decoded sentence: A man was been ared to a proper to a contrice and the reash to a contrice and t

-
Input sentence: Panorama Software, a global provider in proactive business intelligence solutions, recently released NovaView Version 5.5, the newest iteration of its flagship suite of BI applications.
Decoded sentence: A man was been ared to a proper to a contrice and the reash to a contrice and the reash to a contrice and the reash to a contrice and the reash to a contrice and the reash to a contrice an
-
Input sentence: Spanish oil major Repsol YPF plans to boost its investments in Bolivia, Bolivian President Evo Morales said Tuesday during a visit to Spain.
Decoded sentence: A man was been ared to a proper to a contrice and the reash to a contrice and the reash to a contrice and the reash to a contrice and the reash to a contrice and the reash to a contrice an
-
Input sentence: Dollar remittances from overseas Filipinos rose by 7.8 per cent in the first nine months of the year, the Philippine Central Bank said Monday.
Decoded sentence: A man was been ared to a proper to a contric

-
Input sentence: A 16-year-old Warwick teen has been charged with robbing one of his Warwick High School classmates at knifepoint Sunday evening.
Decoded sentence: A man was been ared to a proper to a contrice and the reash to a contrice and the reare the record to a proper to a contrice and the reare the record to a proper to a contrice and the rear
-
Input sentence: Actress Eva Longoria has filed for divorce from basketball star Tony Parker after three years of marriage.
Decoded sentence: A man was been ared to a proper to a contrice and the reash to a contrice and the reare the record to a proper to a contrice and the reare the record to a proper to a contrice and the rear
-
Input sentence: A 30-year-old Jones County man was severely burned in a mobile home fire early this morning in the Powers community.
Decoded sentence: A man was been ared to a proper to a contrice and the reash to a contrice and the reare the record to a proper to a contrice and the reare the record to a proper

-
Input sentence: The Nationalist Party made a huge strategic mistake in taking a position against divorce as it alienated the substantial amount of liberals inside the party, former PN minister Michael Falzon said.
Decoded sentence: A man was been ared to a proper to a contrice and the reash to a contrice and the reare the record to a proper to a contrice and the reare the record to a proper to a contrice and the rear
-
Input sentence: Two men were charged Thursday with smuggling illegal immigrants in a failed attempt to rush the nation's busiest border crossing in vans packed with suspected illegal immigrants.
Decoded sentence: A man was been ared to a proper to a contrice and the reash to a contrice and the reare the record to a proper to a contrice and the reare the record to a proper to a contrice and the rear
-
Input sentence: In a move that could help UK citizens be a bit closer to the political sphere, the BBC has pushed a political website online which will enable people to wa

-
Input sentence: COMMUTERS into London face disruption as London Underground staff start a 24-hour strike today.
Decoded sentence: A man was been ared to a proper to a contrice and the reash to a contrice and the reare the record to a proper to a contrice and the reare the record to a proper to a contrice and the rear
-
Input sentence: Taylor Swift and her One Direction boyfriend Harry Styles' break up is weirdly similar to Justin Bieber and Selena Gomez's split.
Decoded sentence: A man was been ared to a proper to a contrice and the reash to a contrice and the reare the record to a proper to a contrice and the reare the record to a proper to a contrice and the rear
-
Input sentence: To further promote security best practices and standards, NetIQ has joined the Cloud Security Alliance, a not-for-profit organization formed by end users, vendors and other stakeholders to further educate the market on the impact of cloud computing on security and compliance programs.
Decoded sentence: A 

In [34]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=5,
          validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 364s 45ms/step - loss: 0.5863 - val_loss: 0.6005
Epoch 2/5
8000/8000 [==============================] - 310s 39ms/step - loss: 0.5751 - val_loss: 0.5915
Epoch 3/5
8000/8000 [==============================] - 295s 37ms/step - loss: 0.5648 - val_loss: 0.5842
Epoch 4/5
8000/8000 [==============================] - 285s 36ms/step - loss: 0.5552 - val_loss: 0.5758
Epoch 5/5
8000/8000 [==============================] - 288s 36ms/step - loss: 0.5460 - val_loss: 0.5683


In [35]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[-seq_index-2: -seq_index-1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[-seq_index-2])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Mobile website builder Akmin's mobiSiteGalore announced on Monday that it has launched version 4.0 of its mobile website builder solution.
Decoded sentence: The stare of the rest the rest the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of t
-
Input sentence: No. 3 Kentucky and No. 4 Louisville renew their rivalry in the most anticipated yearly game in the Commonwealth on Saturday at Rupp Arena.
Decoded sentence: The stare of the rest the rest the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of t
-
Input sentence: Richard S. Johnson birdied three of his last four holes to shoot a 6-under-par 64 Sunday and win the US Bank Championship for his first US PGA Tour victory.
Decoded sentence: The stare of the rest the rest the stare of the stare of the stare of the stare

-
Input sentence: Panorama Software, a global provider in proactive business intelligence solutions, recently released NovaView Version 5.5, the newest iteration of its flagship suite of BI applications.
Decoded sentence: The stare of the rest the rest the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of t
-
Input sentence: Spanish oil major Repsol YPF plans to boost its investments in Bolivia, Bolivian President Evo Morales said Tuesday during a visit to Spain.
Decoded sentence: The stare of the rest the rest the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of t
-
Input sentence: Dollar remittances from overseas Filipinos rose by 7.8 per cent in the first nine months of the year, the Philippine Central Bank said Monday.
Decoded sentence: The stare of the rest the rest the stare of 

-
Input sentence: A 16-year-old Warwick teen has been charged with robbing one of his Warwick High School classmates at knifepoint Sunday evening.
Decoded sentence: The stare of the rest the rest the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of t
-
Input sentence: Actress Eva Longoria has filed for divorce from basketball star Tony Parker after three years of marriage.
Decoded sentence: The stare of the rest the rest the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of t
-
Input sentence: A 30-year-old Jones County man was severely burned in a mobile home fire early this morning in the Powers community.
Decoded sentence: The stare of the rest the rest the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of 

-
Input sentence: The Nationalist Party made a huge strategic mistake in taking a position against divorce as it alienated the substantial amount of liberals inside the party, former PN minister Michael Falzon said.
Decoded sentence: The stare of the rest the rest the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of t
-
Input sentence: Two men were charged Thursday with smuggling illegal immigrants in a failed attempt to rush the nation's busiest border crossing in vans packed with suspected illegal immigrants.
Decoded sentence: The stare of the rest the rest the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of t
-
Input sentence: In a move that could help UK citizens be a bit closer to the political sphere, the BBC has pushed a political website online which will enable people to wa

-
Input sentence: COMMUTERS into London face disruption as London Underground staff start a 24-hour strike today.
Decoded sentence: The stare of the rest the rest the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of t
-
Input sentence: Taylor Swift and her One Direction boyfriend Harry Styles' break up is weirdly similar to Justin Bieber and Selena Gomez's split.
Decoded sentence: The stare of the rest the rest the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of the stare of t
-
Input sentence: To further promote security best practices and standards, NetIQ has joined the Cloud Security Alliance, a not-for-profit organization formed by end users, vendors and other stakeholders to further educate the market on the impact of cloud computing on security and compliance programs.
Decoded sentence: Th